In [2]:
#Importing essential libraries
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
%matplotlib inline

In [1]:
# Set the paths to your training, validation, and testing folders
train_folder = "Original Images/a. Training Set"
valid_folder = "Original Images/b. Validation Set"
test_folder = "Original Images/c. Testing Set"

# Set the paths to your training, validation, and testing CSV files for labels
train_labels_file = "Groundtruths/training28.csv"
valid_labels_file = "Groundtruths/validation28.csv"
test_labels_file = "Groundtruths/testing28.csv"

In [ ]:
# Load the CSV files for labels
train_labels = pd.read_csv(train_labels_file)
valid_labels = pd.read_csv(valid_labels_file)
test_labels = pd.read_csv(test_labels_file)

In [ ]:
# define constants
img_width, img_height = 100, 100
batch_size = 32
epochs = 10
learning_rate = 0.001

In [ ]:
# Load the CSV file with image filenames and labels
train_df = pd.read_csv(train_labels_file)
valid_df = pd.read_csv(valid_labels_file)
test_df = pd.read_csv(test_labels_file)

# Remove rows from dataframes where filenames are missing
missing_rows = [1506,1505,1744,1752,1753,1756,1762,1770,1781,1788,1792,1842,1846,1873,1878] 
train_df.drop(missing_rows, inplace=True)
missing_rows = [549,550,552,560,590,596]
valid_df.drop(missing_rows, inplace=True)
missing_rows = [553,590,592,564,614,617,622,624,628]
test_df.drop(missing_rows, inplace=True)

# Add the .png extension to the filenames
train_df['ID'] = train_df['ID'].apply(lambda x: str(x) + '.png')
valid_df['ID'] = valid_df['ID'].apply(lambda x: str(x) + '.png')
test_df['ID'] = test_df['ID'].apply(lambda x: str(x) + '.png')

In [ ]:
# Create an ImageDataGenerator object with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5])

# Create flow generators for the training set and validation set
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_folder,
    x_col='ID',
    y_col=train_df.columns[2:],
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='raw')

# Create flow generators for the validation set
valid_datagen = ImageDataGenerator(rescale=1./255)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_df,
    directory=valid_folder,
    x_col='ID',
    y_col=valid_df.columns[2:],
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='raw')


# Create flow generators for the testing set
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_folder,
    x_col='ID',
    y_col=valid_df.columns[2:],
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='raw')

In [3]:
#Model
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(100,100,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(28, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 96, 96, 16)        1216      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 48, 48, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 32)        12832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 22, 22, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 22, 22, 32)        0

In [ ]:
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# train model using fit_generator
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size)

In [ ]:
# get accuracy and loss values
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
y_pred = model.predict_generator(test_generator, steps=test_steps)
precision, recall, thresholds = precision_recall_curve(y_test.ravel(), y_pred.ravel())

# plot precision-recall curve
plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()